In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Sử dụng ChromeDriver
driver = webdriver.Chrome(executable_path="C:\\Users\\TECHCARE\\Desktop\\crawl_project\\chromedriver-win64\\chromedriver.exe")

# Mở trang web
url = 'https://fptshop.com.vn/may-tinh-xach-tay?sort=ban-chay-nhat&trang=9'
driver.get(url)

# Đợi cho các phần tử được tải đầy đủ trong một khoảng thời gian nhất định (10 giây)
wait = WebDriverWait(driver, 10)

# Lấy đường link sản phẩm từ class "cdt-product__name"
product_links = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'cdt-product__name')))

# Lặp qua các phần tử và lấy đường link, sau đó thêm vào danh sách
all_product_links = [link.get_attribute('href') for link in product_links]
print(all_product_links)

# Đóng trình duyệt
driver.quit()

['https://fptshop.com.vn/may-tinh-xach-tay/hp-245-g10-r5-7520u', 'https://fptshop.com.vn/may-tinh-xach-tay/asus-tuf-gaming-fx506hf-hn017w-i5-11400h', 'https://fptshop.com.vn/may-tinh-xach-tay/laptop-dell-inspiron-15-n3520-i5-1235u1-n5i5011w1', 'https://fptshop.com.vn/may-tinh-xach-tay/asus-vivobook-e1404fa-nk186w-r5-7520u', 'https://fptshop.com.vn/may-tinh-xach-tay/asus-tuf-gaming-fx507zc4-hn095w-i5-12500h', 'https://fptshop.com.vn/may-tinh-xach-tay/lenovo-ideapad-3-14iah8-i5-12450h', 'https://fptshop.com.vn/may-tinh-xach-tay/hp-14s-em0086au-r5-7520u', 'https://fptshop.com.vn/may-tinh-xach-tay/hp-pavilion-aero-13-be2098au-r7-7735u', 'https://fptshop.com.vn/may-tinh-xach-tay/asus-vivobook-x1404za-nk387w-i3-1215u', 'https://fptshop.com.vn/may-tinh-xach-tay/acer-aspire-3-a315-44p-r5qg-r7-5700u', 'https://fptshop.com.vn/may-tinh-xach-tay/asus-vivobook-m3504ya-l128w-r5-7530u', 'https://fptshop.com.vn/may-tinh-xach-tay/acer-aspire-7-gaming-a715-76-57cy-i5-12450h', 'https://fptshop.com.vn/may

In [24]:
from selenium.common.exceptions import TimeoutException
def get_product_info(all_product_links):
    driver.get(all_product_links)

    laptop_name = driver.find_element(By.CLASS_NAME, 'st-name').text  #chốt
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    #URL SẢN PHẨM
    product_url = soup.find('meta', {'property': 'og:url'}).get('content')

    #GIÁ VÀ KHUYẾN MÃI 
    try:
        discounted_price = soup.find('div', class_ = 'st-price-main').text.strip().replace('*', '')
    except:
        discounted_price = ''
    try:
        original_price = soup.find('div', class_ = 'st-price-sub').text.strip()
    except:
        original_price = discounted_price
    
    try:
        installment_payment = soup.find('div', class_='btn btn-info btn-xl btn--half').find('strong').text.strip()
    except:
        installment_payment = ''
    
    promotion_variables = []    
    try:
        promotions_list = soup.find('ul', class_='st-boxPromo__list st-boxPromo__link').find_all('label')
        
        for idx, promotion in enumerate(promotions_list, start=1):
            variable_name = f"Promotion_{idx}"
            promotion_text = promotion.text.strip()
            promotion_variables.append((variable_name, promotion_text))
    except AttributeError:
        pass
    #NÚT XEM THÊM
    try:
        see_more_button = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.ic-down-open-big'))
            )
        see_more_button.click()
    except TimeoutException:
        pass
    new_content = driver.page_source
    new_soup = BeautifulSoup(new_content, 'html.parser')
    try:
        promotions_more_list = new_soup.find('ul', class_='st-boxPromo__list st-boxPromo__list--more').find_all('div')
        for idx, promotion in enumerate(promotions_more_list, start=1):
            variable_name = f"Promotion_more_{idx}"
            promotion_text = promotion.text.strip().replace('\xa0Xem chi tiết','')
            promotion_variables.append((variable_name, promotion_text))
    except AttributeError:
        pass
    
    product_price = {
        'product_name': laptop_name,
        'discounted_price': discounted_price,
        'original_price': original_price,
        'installment_payment' : installment_payment,
        'shop_brand': 'FPT shop'
    }
    # Thêm các biến khuyến mại vào dictionary, dự đoán số lượng khuyến mại nhiều nhất là 12
    max_promotions = 12 
    for i in range(1, max_promotions + 1):
        key = f'promotion_{i}'
        # Nếu i vượt quá số lượng thực sự của khuyến mại, thiết lập giá trị là None
        if i <= len(promotion_variables):
            value = promotion_variables[i-1][1]
        else:
            value = ''

        product_price[key] = value
    product_price['product_url'] = product_url

    
    try:
        #Nút Xem chi tiết thông tin sản phẩm
        detail_button = WebDriverWait(driver, 50).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'a.re-link.js--open-modal'))
        )
        detail_button.click()

        # Chờ một khoảng thời gian ngắn để trang chi tiết được tải
        driver.implicitly_wait(10)

        # Lấy HTML của trang đã được tải sau khi nhấp vào nút
        new_html_content = driver.page_source

        # Sử dụng BeautifulSoup
        new_soup_2 = BeautifulSoup(new_html_content, 'html.parser')
        
        img_container = new_soup_2.find('div', class_='swiper-slide swiper-slide-active')
        if img_container:
            img_tag = img_container.find('img')
            if img_tag and 'src' in img_tag.attrs:
                image_url = img_tag['src']
            else:
                image_url = ""
            
        else: 
            image_url = ""
        list_items = new_soup_2.find_all('li')

        # Khởi tạo biến với giá trị mặc định
        laptop_brand = None
        source = None
        release_date = None
        warranty = None
        part_number = None
        # Duyệt qua từng thẻ <li> và kiểm tra xem có chứa thông tin nào không
        for item in list_items:
            text = item.text.strip()
            if 'Thương hiệu' in text:
                laptop_brand = item.find('div').text.strip()
            elif 'Xuất xứ' in text:
                source = item.find('div').text.strip()
            elif 'Thời điểm ra mắt' in text:
                release_date = item.find('div').text.strip()
            elif 'Thời gian bảo hành' in text:
                warranty = item.find('div').text.strip()
            elif 'P/N' in text: 
                part_number = item.find('div').text.strip()
        table_rows = new_soup_2.find_all('table', class_='st-table')  # Lấy tất cả các bảng
        info = {}
        for table in table_rows:
            rows = table.find('tbody').find_all('tr')
            for row in rows:
                columns = row.find_all('td')
                key = columns[0].text.strip()
                value = columns[1].text.strip()
                # Gán giá trị vào dictionary
                info[key] = value
        size = info.get('Kích thước', '')
        weight = info.get('Trọng lượng sản phẩm', '')
        color = info.get('Màu sắc', '')
        cooling = info.get('Tản nhiệt', '')

        material = new_soup_2.find('div', id='data-1').find('ul', class_='st-list')
        laptop_material_list = [item.text.strip() for item in material.find_all('li')]
        laptop_material = ', '.join(laptop_material_list)
        CPU_brand = info.get('Hãng CPU', '')
        CPU_tech = info.get('Công nghệ CPU', '')
        CPU_type = info.get('Loại CPU', '')
        CPU_speed = info.get('Tốc độ CPU', '')
        CPU_speed_max = info.get('Tốc độ tối đa', '')
        RAM_capacity = info.get('Dung lượng RAM', '')
        RAM_type = info.get('Loại RAM', '')
        RAM_speed = info.get('Tốc độ RAM', '')
        Max_RAM_sp = info.get('Hỗ trợ RAM tối đa', '')
        screen_size = info.get('Kích thước màn hình', '')
        screen_tech = info.get('Công nghệ màn hình', '')
        screen_resolution = info.get('Độ phân giải', '')
        screen_scanning_freq= info.get('Tần số quét', '')
        screen_background_panel = info.get('Tấm nền', '')
        card_types = new_soup_2.find('div', id='data-5').select('.st-table-wrapper .st-table-title')

        # Khởi tạo các danh sách để lưu thông tin
        card_type_list = []
        card_brand_list = []
        card_model_list = []

        # Lặp qua danh sách các loại card và lấy thông tin cụ thể
        for card_type in card_types:
            # Lấy tên của loại card (Card onboard hoặc Card rời)
            card_type_name = card_type.text.strip()
            if card_type_name != "Card onboard" and card_type_name != "Card rời":
                card_type_name = ""
            card_type_list.append(card_type_name)
            # Lấy thông tin cụ thể của loại card
            table_rows = card_type.find_next('table').find('tbody').find_all('tr')
            card_info = {}
            for row in table_rows:
                columns = row.find_all('td')
                key = columns[0].text.strip()
                value = columns[1].text.strip()
                card_info[key] = value

            # Lấy thông tin về hãng và model
            c_brand = card_info.get('Hãng', '')
            c_model = card_info.get('Model', '')
            card_brand_list.append(c_brand)
            card_model_list.append(c_model)

        # Chuyển danh sách thành chuỗi nếu cần thiết
        card_type = ', '.join(card_type_list)
        card_brand = ', '.join(card_brand_list)
        card_model = ', '.join(card_model_list)

        hard_driver_type = info.get('Kiểu ổ cứng', '')
        hard_driver_model = info.get('Loại SSD', '')
        hard_driver_memory = info.get('Dung lượng', '')
        port_table = new_soup_2.find('div', id='data-8').find('ul', class_='st-list')
        if port_table:
            communication_ports_list = [item.text.strip() for item in port_table.find_all('li')]
            communication_ports = ', '.join(communication_ports_list)
        else:
            port_table = new_soup_2.find('div', id='data-7').find('ul', class_='st-list')
            communication_ports_list = [item.text.strip() for item in port_table.find_all('li')]
            communication_ports = ', '.join(communication_ports_list)

        keyboard_type = info.get('Kiểu bàn phím', '')
        keyboard_num = info.get('Bàn phím số', '')
        keyboard_light = info.get('Đèn bàn phím', '')
        keyboard_touchpad = info.get('TouchPad', '')
        baterry_capacity = info.get('Dung lượng pin', '')
        operating_system = info.get('OS', '')

        product_info = {
            'product_id': part_number,
            'product_name': laptop_name,
            'product_brand': laptop_brand,
            'product_size': size,
            'product_color': color,
            'product_weight': weight,
            'source': source,
            'release_date': release_date,
            'warranty': warranty,
            'product_cooling': cooling,
            'product_material': laptop_material,
            'CPU_brand': CPU_brand,
            'CPU_tech': CPU_tech,
            'CPU_type': CPU_type,
            'CPU_speed': CPU_speed,
            'CPU_speed_max': CPU_speed_max,
            'RAM_capacity': RAM_capacity,
            'RAM_type': RAM_type,
            'RAM_speed': RAM_speed,
            'RAM_max_support': Max_RAM_sp,
            'screen_size': screen_size,
            'screen_tech': screen_tech,
            'screen_resolution': screen_resolution,
            'screen_scanning_freq': screen_scanning_freq,
            'screen_background_panel': screen_background_panel,
            'card_type': card_type,
            'card_brand': card_brand,
            'card_model': card_model,       
            'hard_driver_type': hard_driver_type,
            'hard_driver_model': hard_driver_model,
            'hard_driver_memory': hard_driver_memory,
            'communication_ports': communication_ports,
            'keyboard_type': keyboard_type,
            'keyboard_num': keyboard_num,
            'keyboard_light': keyboard_light,
            'keyboard_touchpad': keyboard_touchpad,
            'baterry_capacity': baterry_capacity,
            'operating_system': operating_system,
            'image_url': image_url
        }

        return product_info, product_price
    except TimeoutException:
        return None
# Mở trình duyệt
driver = webdriver.Chrome(executable_path="C:\\Users\\TECHCARE\\Desktop\\crawl_project\\chromedriver-win64\\chromedriver.exe")
# Duyệt qua các đường link sản phẩm và lấy thông tin
data = []
data_price = []
for product_link in all_product_links:
    result = get_product_info(product_link)
    if result is not None:
        product_info, product_price = result
    else:
        continue
    data.append(product_info)
    data_price.append(product_price)
    print(product_info)
    print(product_price)
# Đóng trình duyệt
driver.quit()

{'product_id': '8F154PA', 'product_name': 'Laptop HP 245 G10 R5 7520U/8GB/256GB/14"FHD/Win11 (8F154PA) (No.00900029)', 'product_brand': 'HP', 'product_size': '32.4 x 21.5 x 1.79 cm', 'product_color': 'Bạc', 'product_weight': '1.36 kg', 'source': 'Trung Quốc', 'release_date': '2023', 'warranty': '12', 'product_cooling': '', 'product_material': 'Nhựa, Vỏ máy: Nhựa', 'CPU_brand': 'AMD', 'CPU_tech': 'Ryzen 5', 'CPU_type': '7520U', 'CPU_speed': '4.30 GHz Ghz', 'CPU_speed_max': '', 'RAM_capacity': '8 GB', 'RAM_type': '', 'RAM_speed': '', 'RAM_max_support': 'Không nâng cấp', 'screen_size': '14.0 inch', 'screen_tech': 'LCD', 'screen_resolution': '1920 x 1080 Pixels', 'screen_scanning_freq': '60 Hz', 'screen_background_panel': 'IPS', 'card_type': 'Card onboard', 'card_brand': 'AMD', 'card_model': 'Radeon', 'hard_driver_type': 'SSD', 'hard_driver_model': 'M.2 NVMe', 'hard_driver_memory': '256 GB', 'communication_ports': '1 HDMI 1.4, HDMI 1.4', 'keyboard_type': 'English International Backlit Keyb

In [26]:
import pandas as pd
df = pd.DataFrame(data)
df

,product_id,product_name,product_brand,product_size,product_color,product_weight,source,release_date,warranty,product_cooling,...,hard_driver_model,hard_driver_memory,communication_ports,keyboard_type,keyboard_num,keyboard_light,keyboard_touchpad,baterry_capacity,operating_system,image_url
0,8F154PA,"Laptop HP 245 G10 R5 7520U/8GB/256GB/14""FHD/Wi...",HP,32.4 x 21.5 x 1.79 cm,Bạc,1.36 kg,Trung Quốc,2023,12,,...,M.2 NVMe,256 GB,"1 HDMI 1.4, HDMI 1.4",English International Backlit Keyboard,Không,Không,Multi-touch touchpad,3 Cell,Windows,https://images.fpt.shop/unsafe/fit-in/585x390/...
1,FX506HF-HN017W,Laptop Asus TUF Gaming FX506HF-HN017W i5 11400...,Asus,359 x 256 x 22.8 ~ 24.3 mm,Đen,2.3 kg,Trung Quốc,4/2023,24,,...,M2. PCIe,512 GB,"1 HDMI 2.1, 1 Jack 3.5 mm, 1 LAN, 1 Thunderbol...",Backlit Chiclet Keyboard,Có,LED,Multi-touch touchpad,3 Cell,Windows,https://images.fpt.shop/unsafe/fit-in/585x390/...
2,N5I5011W1,Laptop Dell Inspiron 15 N3520 i5 1235U/16GB/51...,Dell,35.85 x 23.49 x 1.75 cm,Đen,1.9 kg,Trung Quốc,2022,12,,...,M.2 NVMe 1,512 GB,"1 HDMI, 1 USB 2.0, 2 USB 3.2 Gen 1 Type-A",English International Non-backlit Keyboard,Có,Không,Touchpad supports four-finger gestures,,Windows,https://images.fpt.shop/unsafe/fit-in/585x390/...
3,90NB0ZS2-M00730,Laptop Asus Vivobook E1404FA-NK186W R5 7520U/1...,Asus,32.45 x 21.39 x 1.79 cm,Đen,1.3 kg,Trung Quốc,4/2023,24,1 quạt,...,M2. PCIe 1,512 GB,"1 DC-in jack, 1 HDMI 1.4, 1 Jack 3.5 mm, 1 USB...",Chiclet Keyboard,Không,Không,Multi-touch touchpad,42 Wh,Windows,https://images.fpt.shop/unsafe/fit-in/585x390/...
4,90NR0GW2-M00670,Laptop Asus TUF Gaming FX507ZC4-HN095W i5 1250...,Asus,35.4 x 25.1 x 2.24 ~ 2.49 cm,Xám,2.2 kg,Trung Quốc,2022,24,2 quạt,...,M2. PCIe 2,512 GB,"1 HDMI 2.1, 1 LAN, 1 Thunderbolt, 1 USB 3.2 Ge...",Backlit Chiclet Keyboard,Có,LED,Multi-touch touchpad,56 Wh,Windows,https://images.fpt.shop/unsafe/fit-in/585x390/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,9Q972PA,,HP,35.98 x 23.6 x 1.86 cm,Bạc,1.59 kg,Trung Quốc,2023,12,,...,M.2 NVMe 1,512 GB,2 Type C,"6-row, multimedia Fn keys, numeric keypad",Có,Không,Multi-touch touchpad,41 Wh,Windows,https://images.fpt.shop/unsafe/fit-in/585x390/...
222,82R900ECVN,,Lenovo,313.1 x 224.9 x 17.8 mm,Xám,1.55 kg,Trung Quốc,2023,24,,...,M2. PCIe,512 GB,"1 HDMI 1.4, 1 Jack 3.5 mm, 1 Power connector, ...",Backlit Chiclet Keyboard,Không,Không,Multi-touch touchpad,52.5 Wh,Windows,https://images.fpt.shop/unsafe/fit-in/585x390/...
223,83EQ0009VN,,Lenovo,324.3 x 213.8 x 17.9 mm,Xám,1.37 kg,Trung Quốc,2023,24,,...,M.2 NVMe 1,1 TB,"1 Card reader, 1 HDMI 1.4, 1 Jack 3.5 mm, 1 US...",English International Non-backlit Keyboard,Không,LED,Multi-touch touchpad,47 Wh,Windows,https://images.fpt.shop/unsafe/fit-in/585x390/...
224,82R900EDVN,,Lenovo,313.1 x 224.9 x 17.8 mm,Xám,1.55 kg,Trung Quốc,2023,24,,...,M2. PCIe,512 GB,"1 HDMI 1.4, 1 Jack 3.5 mm, 1 Power connector, ...",Backlit Chiclet Keyboard,Không,Không,Multi-touch touchpad,52.5 Wh,Windows,https://images.fpt.shop/unsafe/fit-in/585x390/...


In [27]:
dt = pd.DataFrame(data_price)
dt

,product_name,discounted_price,original_price,installment_payment,shop_brand,promotion_1,promotion_2,promotion_3,promotion_4,promotion_5,promotion_6,promotion_7,promotion_8,promotion_9,promotion_10,promotion_11,promotion_12,product_url
0,"Laptop HP 245 G10 R5 7520U/8GB/256GB/14""FHD/Wi...",10.190.000₫,13.290.000₫,TRẢ GÓP 0%,FPT shop,"Giảm ngay 3,100,000đ áp dụng đến 25/03","Giảm ngay 700,000đ Trả góp 0%","Tặng PMH 400,000đ mua LCD MSI, Viewsonic, Xiao...",Ưu đãi đặc quyền dành cho Học sinh - Sinh viên...,Giảm lên đến 25% khi mua gói bảo hành rơi vỡ/m...,Tặng Balo Laptop,Tặng tai nghe choàng đầu ProOne trị giá 449.00...,"Giảm thêm 100,000đ khi mua trả góp qua FE CREDIT",,,,,https://fptshop.com.vn/may-tinh-xach-tay/hp-24...
1,Laptop Asus TUF Gaming FX506HF-HN017W i5 11400...,17.490.000₫,19.990.000₫,TRẢ GÓP 0%,FPT shop,"Giảm ngay 2,500,000đ áp dụng đến 25/03","Giảm ngay 1,000,000đ Trả góp 0%","Tặng PMH 400,000đ mua LCD MSI, Viewsonic, Xiao...",Ưu đãi đặc quyền dành cho Học sinh - Sinh viên...,Giảm lên đến 25% khi mua gói bảo hành rơi vỡ/m...,Tặng Balo Laptop,Tặng tai nghe choàng đầu ProOne trị giá 449.00...,"Giảm thêm 100,000đ khi mua trả góp qua FE CREDIT","Tặng voucher 399,000đ mua Chuột iCore GM03",,,,https://fptshop.com.vn/may-tinh-xach-tay/asus-...
2,Laptop Dell Inspiron 15 N3520 i5 1235U/16GB/51...,15.490.000₫,16.690.000₫,TRẢ GÓP 0%,FPT shop,"Giảm ngay 1,200,000đ áp dụng đến 25/03",Trả góp 0%,"Tặng PMH 400,000đ mua LCD MSI, Viewsonic, Xiao...",Ưu đãi đặc quyền dành cho Học sinh - Sinh viên...,Giảm lên đến 25% khi mua gói bảo hành rơi vỡ/m...,Tặng Balo Laptop,Tặng tai nghe choàng đầu ProOne trị giá 449.00...,"Giảm thêm 100,000đ khi mua trả góp qua FE CREDIT",,,,,https://fptshop.com.vn/may-tinh-xach-tay/lapto...
3,Laptop Asus Vivobook E1404FA-NK186W R5 7520U/1...,12.490.000₫,14.990.000₫,TRẢ GÓP 0%,FPT shop,"Giảm ngay 2,500,000đ áp dụng đến 25/03",Trả góp 0%,"Tặng PMH 400,000đ mua LCD MSI, Viewsonic, Xiao...",Ưu đãi đặc quyền dành cho Học sinh - Sinh viên...,Giảm lên đến 25% khi mua gói bảo hành rơi vỡ/m...,Tặng Balo Laptop,Tặng tai nghe choàng đầu ProOne trị giá 449.00...,"Giảm thêm 100,000đ khi mua trả góp qua FE CREDIT",,,,,https://fptshop.com.vn/may-tinh-xach-tay/asus-...
4,Laptop Asus TUF Gaming FX507ZC4-HN095W i5 1250...,19.490.000₫,24.990.000₫,TRẢ GÓP 0%,FPT shop,"Giảm ngay 5,500,000đ áp dụng đến 25/03","Giảm ngay 2,000,000đ Trả góp 0%","Tặng PMH 400,000đ mua LCD MSI, Viewsonic, Xiao...",Ưu đãi đặc quyền dành cho Học sinh - Sinh viên...,Giảm lên đến 25% khi mua gói bảo hành rơi vỡ/m...,Tặng Balo Laptop,Tặng tai nghe choàng đầu ProOne trị giá 449.00...,"Giảm thêm 100,000đ khi mua trả góp qua FE CREDIT","Tặng voucher 399,000đ mua Chuột iCore GM03",,,,https://fptshop.com.vn/may-tinh-xach-tay/asus-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,,,,,FPT shop,"Tặng PMH 400,000đ mua LCD MSI, Viewsonic, Xiao...",Ưu đãi đặc quyền dành cho Học sinh - Sinh viên...,Giảm lên đến 25% khi mua gói bảo hành rơi vỡ/m...,Tặng Balo Laptop,Tặng tai nghe choàng đầu ProOne trị giá 449.00...,"Giảm thêm 100,000đ khi mua trả góp qua FE CREDIT",,,,,,,https://fptshop.com.vn/may-tinh-xach-tay/hp-15...
222,,,,,FPT shop,"Tặng PMH 400,000đ mua LCD MSI, Viewsonic, Xiao...",Ưu đãi đặc quyền dành cho Học sinh - Sinh viên...,Giảm lên đến 25% khi mua gói bảo hành rơi vỡ/m...,Tặng Balo Laptop,Tặng tai nghe choàng đầu ProOne trị giá 449.00...,"Giảm thêm 100,000đ khi mua trả góp qua FE CREDIT",,,,,,,https://fptshop.com.vn/may-tinh-xach-tay/lenov...
223,,,,,FPT shop,"Tặng PMH 400,000đ mua LCD MSI, Viewsonic, Xiao...",Ưu đãi đặc quyền dành cho Học sinh - Sinh viên...,Giảm lên đến 25% khi mua gói bảo hành rơi vỡ/m...,Tặng Balo Laptop,Tặng tai nghe choàng đầu ProOne trị giá 449.00...,"Giảm thêm 100,000đ khi mua trả góp qua FE CREDIT",,,,,,,https://fptshop.com.vn/may-tinh-xach-tay/lenov...
224,,,,,FPT shop,"Tặng PMH 400,000đ mua LCD MSI, Viewsonic, Xiao...",Ưu đãi đặc quyền dành cho Học sinh - Sinh viên...,Giảm lên đến 25% khi m

In [28]:
dt['product_id'] = df['product_id']
dt

,product_name,discounted_price,original_price,installment_payment,shop_brand,promotion_1,promotion_2,promotion_3,promotion_4,promotion_5,promotion_6,promotion_7,promotion_8,promotion_9,promotion_10,promotion_11,promotion_12,product_url,product_id
0,"Laptop HP 245 G10 R5 7520U/8GB/256GB/14""FHD/Wi...",10.190.000₫,13.290.000₫,TRẢ GÓP 0%,FPT shop,"Giảm ngay 3,100,000đ áp dụng đến 25/03","Giảm ngay 700,000đ Trả góp 0%","Tặng PMH 400,000đ mua LCD MSI, Viewsonic, Xiao...",Ưu đãi đặc quyền dành cho Học sinh - Sinh viên...,Giảm lên đến 25% khi mua gói bảo hành rơi vỡ/m...,Tặng Balo Laptop,Tặng tai nghe choàng đầu ProOne trị giá 449.00...,"Giảm thêm 100,000đ khi mua trả góp qua FE CREDIT",,,,,https://fptshop.com.vn/may-tinh-xach-tay/hp-24...,8F154PA
1,Laptop Asus TUF Gaming FX506HF-HN017W i5 11400...,17.490.000₫,19.990.000₫,TRẢ GÓP 0%,FPT shop,"Giảm ngay 2,500,000đ áp dụng đến 25/03","Giảm ngay 1,000,000đ Trả góp 0%","Tặng PMH 400,000đ mua LCD MSI, Viewsonic, Xiao...",Ưu đãi đặc quyền dành cho Học sinh - Sinh viên...,Giảm lên đến 25% khi mua gói bảo hành rơi vỡ/m...,Tặng Balo Laptop,Tặng tai nghe choàng đầu ProOne trị giá 449.00...,"Giảm thêm 100,000đ khi mua trả góp qua FE CREDIT","Tặng voucher 399,000đ mua Chuột iCore GM03",,,,https://fptshop.com.vn/may-tinh-xach-tay/asus-...,FX506HF-HN017W
2,Laptop Dell Inspiron 15 N3520 i5 1235U/16GB/51...,15.490.000₫,16.690.000₫,TRẢ GÓP 0%,FPT shop,"Giảm ngay 1,200,000đ áp dụng đến 25/03",Trả góp 0%,"Tặng PMH 400,000đ mua LCD MSI, Viewsonic, Xiao...",Ưu đãi đặc quyền dành cho Học sinh - Sinh viên...,Giảm lên đến 25% khi mua gói bảo hành rơi vỡ/m...,Tặng Balo Laptop,Tặng tai nghe choàng đầu ProOne trị giá 449.00...,"Giảm thêm 100,000đ khi mua trả góp qua FE CREDIT",,,,,https://fptshop.com.vn/may-tinh-xach-tay/lapto...,N5I5011W1
3,Laptop Asus Vivobook E1404FA-NK186W R5 7520U/1...,12.490.000₫,14.990.000₫,TRẢ GÓP 0%,FPT shop,"Giảm ngay 2,500,000đ áp dụng đến 25/03",Trả góp 0%,"Tặng PMH 400,000đ mua LCD MSI, Viewsonic, Xiao...",Ưu đãi đặc quyền dành cho Học sinh - Sinh viên...,Giảm lên đến 25% khi mua gói bảo hành rơi vỡ/m...,Tặng Balo Laptop,Tặng tai nghe choàng đầu ProOne trị giá 449.00...,"Giảm thêm 100,000đ khi mua trả góp qua FE CREDIT",,,,,https://fptshop.com.vn/may-tinh-xach-tay/asus-...,90NB0ZS2-M00730
4,Laptop Asus TUF Gaming FX507ZC4-HN095W i5 1250...,19.490.000₫,24.990.000₫,TRẢ GÓP 0%,FPT shop,"Giảm ngay 5,500,000đ áp dụng đến 25/03","Giảm ngay 2,000,000đ Trả góp 0%","Tặng PMH 400,000đ mua LCD MSI, Viewsonic, Xiao...",Ưu đãi đặc quyền dành cho Học sinh - Sinh viên...,Giảm lên đến 25% khi mua gói bảo hành rơi vỡ/m...,Tặng Balo Laptop,Tặng tai nghe choàng đầu ProOne trị giá 449.00...,"Giảm thêm 100,000đ khi mua trả góp qua FE CREDIT","Tặng voucher 399,000đ mua Chuột iCore GM03",,,,https://fptshop.com.vn/may-tinh-xach-tay/asus-...,90NR0GW2-M00670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,,,,,FPT shop,"Tặng PMH 400,000đ mua LCD MSI, Viewsonic, Xiao...",Ưu đãi đặc quyền dành cho Học sinh - Sinh viên...,Giảm lên đến 25% khi mua gói bảo hành rơi vỡ/m...,Tặng Balo Laptop,Tặng tai nghe choàng đầu ProOne trị giá 449.00...,"Giảm thêm 100,000đ khi mua trả góp qua FE CREDIT",,,,,,,https://fptshop.com.vn/may-tinh-xach-tay/hp-15...,9Q972PA
222,,,,,FPT shop,"Tặng PMH 400,000đ mua LCD MSI, Viewsonic, Xiao...",Ưu đãi đặc quyền dành cho Học sinh - Sinh viên...,Giảm lên đến 25% khi mua gói bảo hành rơi vỡ/m...,Tặng Balo Laptop,Tặng tai nghe choàng đầu ProOne trị giá 449.00...,"Giảm thêm 100,000đ khi mua trả góp qua FE CREDIT",,,,,,,https://fptshop.com.vn/may-tinh-xach-tay/lenov...,82R900ECVN
223,,,,,FPT shop,"Tặng PMH 400,000đ mua LCD MSI, Viewsonic, Xiao...",Ưu đãi đặc quyền dành cho Học sinh - Sinh viên...,Giảm lên đến 25% khi mua gói bảo hành rơi vỡ/m...,Tặng Balo Laptop,Tặng tai nghe choàng đầu ProOne trị giá 449.00...,"Giảm thêm 100,000đ khi mua trả góp qua FE CREDIT",,,,,,,https://fptshop.com.vn/may-tinh-xach-tay/lenov...,83EQ0009VN
224,,,,,FPT shop,"Tặng PMH 400,

In [29]:
df.to_excel('product_info.xlsx', index=False)
dt.to_excel('fpt_price.xlsx', index = False)